# Spark Structured API ou Spark SQL

Spark SQL est un module Spark pour le traitement structuré des données. Contrairement à l'API Spark RDD de base, les interfaces fournies par Spark SQL fournissent à Spark plus d'informations sur la structure des données et sur le calcul effectué. En interne, Spark SQL utilise ces informations supplémentaires pour effectuer des optimisations supplémentaires. Il existe plusieurs façons d'interagir avec Spark SQL, y compris SQL et l'API Dataset. Lors du calcul d'un résultat, le même moteur d'exécution est utilisé, quelle que soit l'API/langage utilisée pour exprimer le calcul. Cette unification signifie que les développeurs peuvent facilement passer d'une API à l'autre, ce qui constitue la manière la plus naturelle d'exprimer une transformation donnée.

## SQL

L'une des utilisations de Spark SQL est l'exécution de requêtes SQL. Spark SQL peut également être utilisé pour lire des données à partir d'une installation de ruche existante. Lorsque vous exécutez SQL à partir d'un autre langage de programmation, les résultats seront renvoyés sous forme de Dataset/DataFrame. Vous pouvez également interagir avec l'interface SQL en utilisant la ligne de commande ou via JDBC/ODBC.

## Datasets

Un Dataset est une collection de données distribuées. Le Dataset est une nouvelle interface ajoutée dans Spark 1.6 qui offre les avantages des RDD (typage fort, possibilité d'utiliser des fonctions lambda puissantes) avec les avantages du moteur d'exécution optimisé de Spark SQL. Un Dataset peut être construit à partir d'objets JVM et ensuite manipulé à l'aide de transformations fonctionnelles (map, flatMap, filtre, etc.). L'API du Dataset est disponible en Scala et Java. Python ne supporte pas l'API Dataset. Mais en raison de la nature dynamique de Python, de nombreux avantages de l'API Dataset sont déjà disponibles (c'est-à-dire que vous pouvez accéder au champ d'une ligne par son nom naturellement row.columnName). Le cas de R est similaire.

## DataFrames

Une DataFrame est un Dataset organisé en colonnes nommées. Il est conceptuellement équivalent à une table dans une base de données relationnelle ou à une trame de données en R/Python, mais avec des optimisations plus riches sous le capot. Les DataFrames peuvent être construites à partir d'un large éventail de sources telles que : des fichiers de données structurées, des tables dans une ruche, des bases de données externes ou des RDD existants. L'API DataFrame est disponible en Scala, Java, Python et R. En Scala et Java, une DataFrame est représentée par un ensemble de lignes de données. Dans l'API Scala, la DataFrame est simplement un alias de type Dataset[Row]. Alors que, dans l'API Java, les utilisateurs doivent utiliser Dataset<Row> pour représenter une DataFrame.

### Creer un DataFrames

In [4]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.9` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                // Not required since almond 0.7.0 (will be automatically added when importing spark)

In [5]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [6]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@5e68f294

Vous pouvez importer la bibliothèque `spark implicits` et créer un DataFrame avec la méthode `toDF()`.

In [7]:
import spark.implicits._

val df = Seq(
    ("Boston", "USA", 0.67), 
    ("Dubai", "UAE", 3.1), 
    ("Dakar", "Senegal", 5.28)
).toDF("city", "country", "population")

import spark.implicits._


df: DataFrame = [city: string, country: string ... 1 more field]

Vous pouvez visialiser le contenu d'un DataFrame via la methode `show()`:

In [8]:
df.show()

+------+-------+----------+
|  city|country|population|
+------+-------+----------+
|Boston|    USA|      0.67|
| Dubai|    UAE|       3.1|
| Dakar|Senegal|      5.28|
+------+-------+----------+



### Ajouter des colonnes

Des colonnes peuvent être ajoutées à un DataFrame avec la méthode `withColumn()`.

Ajoutons une colonne `is_big_city` au DataFrame qui renvoie vrai si la ville contient plus de
un million de personnes:

In [9]:
import org.apache.spark.sql.functions.col

val df2 = df.withColumn("is_big_city", col("population") > 1)
df2.show()

+------+-------+----------+-----------+
|  city|country|population|is_big_city|
+------+-------+----------+-----------+
|Boston|    USA|      0.67|      false|
| Dubai|    UAE|       3.1|       true|
| Dakar|Senegal|      5.28|       true|
+------+-------+----------+-----------+



import org.apache.spark.sql.functions.col


df2: DataFrame = [city: string, country: string ... 2 more fields]

Les DataFrames sont immuables, de sorte que la méthode `withColumn()` renvoie un nouveau DataFrame. `withColumn()` ne fait pas muter le DataFrame original. 

Confirmons-nous que df est toujours le même avec `df.show()`.

In [10]:
df.show()

+------+-------+----------+
|  city|country|population|
+------+-------+----------+
|Boston|    USA|      0.67|
| Dubai|    UAE|       3.1|
| Dakar|Senegal|      5.28|
+------+-------+----------+



df ne contient pas la colonne `is_big_city`, nous avons donc confirmé que `withColumn()` n'a pas fait muter df.

### Filtrer des lignes

La méthode `filter()` supprime les lignes d'un DataFrame:

In [11]:
df.filter(col("population") > 1).show()

+-----+-------+----------+
| city|country|population|
+-----+-------+----------+
|Dubai|    UAE|       3.1|
|Dakar|Senegal|      5.28|
+-----+-------+----------+



Il est un peu difficile de lire un code avec plusieurs appels de méthode sur la même ligne, alors découpons ce code sur plusieurs lignes:

In [12]:
df
 .filter(col("population") > 1)
 .show()

+-----+-------+----------+
| city|country|population|
+-----+-------+----------+
|Dubai|    UAE|       3.1|
|Dakar|Senegal|      5.28|
+-----+-------+----------+



Nous pouvons également affecter la DataFrame filtrée à une variable distincte plutôt que d'enchaîner les appels de méthode:

In [13]:
val filteredDF = df.filter(col("population") > 1)
filteredDF.show()

+-----+-------+----------+
| city|country|population|
+-----+-------+----------+
|Dubai|    UAE|       3.1|
|Dakar|Senegal|      5.28|
+-----+-------+----------+



filteredDF: Dataset[Row] = [city: string, country: string ... 1 more field]

### Schema

Le schema d'un DataFrame peut être imprimé sur la console avec la méthode `printSchema()`. 

In [14]:
df.printSchema

root
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- population: double (nullable = false)



La méthode schema renvoie une représentation codée du `schema` DataFrame:

In [15]:
df.schema

res14: types.StructType = StructType(
  StructField("city", StringType, true, {}),
  StructField("country", StringType, true, {}),
  StructField("population", DoubleType, false, {})
)

Chaque colonne d'un Spark DataFrame est modélisée comme un objet StructField avec un nom, un columnType et des propriétés "nullables". L'ensemble du schéma de la DataFrame est modélisé comme un `StructType`, qui est une collection d'objets `StructField`.

Créons un schéma pour un DataFrame qui comporte des colonnes `first_name` et `age`:

In [16]:
import org.apache.spark.sql.types._

StructType(
    Seq(
        StructField("first_name", StringType, true),
        StructField("age", DoubleType, true)
    )
)

import org.apache.spark.sql.types._


res15_1: StructType = StructType(
  StructField("first_name", StringType, true, {}),
  StructField("age", DoubleType, true, {})
)

L'interface de programmation de Spark permet de définir facilement le schéma exact que vous souhaitez pour vos DataFrames.

### Créer des DataFrames avec createDataFrame()

La méthode `toDF()` pour créer des DataFrames est rapide, mais elle est limitée car elle ne vous permet pas de définir votre schéma (elle déduit le schéma pour vous). La méthode createDataFrame() vous permet de définir le schéma de votre DataFrame:

In [17]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.Row

val animalData = Seq(
    Row(30, "bat"),
    Row(2, "mouse"),
    Row(25, "horse")
)

val animalSchema = List(
    StructField("average_lifespan", IntegerType, true), 
    StructField("animal_type", StringType, true)
)

val animalDF = spark.createDataFrame(
    spark.sparkContext.parallelize(animalData), 
    StructType(animalSchema)
)

import org.apache.spark.sql.types._ 

import org.apache.spark.sql.Row


animalData: Seq[Row] = List([30,bat], [2,mouse], [25,horse])
animalSchema: List[StructField] = List(
  StructField("average_lifespan", IntegerType, true, {}),
  StructField("animal_type", StringType, true, {})
)
animalDF: DataFrame = [average_lifespan: int, animal_type: string]

In [18]:
animalDF.show()

show at cmd17.sc:1

1 / 1

show at cmd17.sc:1

3 / 3

+----------------+-----------+
|average_lifespan|animal_type|
+----------------+-----------+
|              30|        bat|
|               2|      mouse|
|              25|      horse|
+----------------+-----------+



Nous pouvons utiliser la méthode `animalDF.printSchema()` pour confirmer que le schéma a été créé comme spécifié:

In [19]:
animalDF.printSchema()

root
 |-- average_lifespan: integer (nullable = true)
 |-- animal_type: string (nullable = true)



Les DataFrames sont les éléments fondamentaux de Spark. Toutes les analyses d'apprentissage machine et de streaming sont construites sur l'API DataFrame.

Voyons maintenant comment construire des fonctions pour manipuler les DataFrames.

## Travailler avec des fichiers CSV

Les fichiers CSV sont parfaits pour apprendre Spark.
Lorsque vous construisez de gros systèmes de données, vous voudrez généralement utiliser un format de fichier plus sophistiqué comme Parquet ou Avro, mais nous utiliserons généralement les CSV dans ce cours car ils sont lisibles par l'homme.
Une fois que vous avez appris à utiliser les fichiers CSV, il est facile d'utiliser d'autres formats de fichiers.

### Lecture d'un fichier CSV dans un DataFrame

Créons un fichier CSV avec ce chemin : `data/cat_data/inputs/file1.csv`. Le fichier doit contenir ces données :

In [20]:
val path = "data/cat_data/inputs/file1.csv"
val df = spark
        .read
        .option("header","true")
        .csv(path)

csv at cmd19.sc:5

1 / 1

path: String = "data/cat_data/inputs/file1.csv"
df: DataFrame = [cat_name: string, cat_age: string]

Affichons le contenu du DataFrame :

In [21]:
df.show()

show at cmd20.sc:1

1 / 1

+--------+-------+
|cat_name|cat_age|
+--------+-------+
|  fluffy|      4|
|    spot|      3|
+--------+-------+



Examinons également le schema du DataFrame :

In [22]:
df.printSchema

root
 |-- cat_name: string (nullable = true)
 |-- cat_age: string (nullable = true)



Spark en déduit que les colonnes sont des chaînes de caractères.
Vous pouvez également définir manuellement le schéma d'un CSV lors de son chargement dans un DataFrame.
Dans la suite du cours, nous expliquerons comment demander à Spark de charger la colonne cat_age en tant qu'entier.

### Ecriture d'un DataFrame sur disque

Ajoutons une colonne `speak` au DataFrame et écrivons les données sur le disque:

import org.apache.spark.sql.functions.lit

df
  .withColumn("speak", lit("meow"))
  .write
  .mode("overwrite")
  .csv("data/cat_data/outputs/cat_output1")

## Les méthodes de colonnes

La classe Spark Column définit une variété de méthodes de colonnes pour manipuler les DataFrames.
Cette section montre comment instancier les objets Column et comment utiliser les plus importantes méthodes de colonnes.

### Un exemple simple 

Créons un DataFrame avec les superhéros et leur ville d'origine:

In [20]:
val df = Seq(
    ("thor", "new york"), 
    ("aquaman", "atlantis"), 
    ("wolverine", "new york")
).toDF("superhero","city")

df: DataFrame = [superhero: string, city: string]

Utilisons la méthode `startsWith()` pour identifier toutes les villes qui commencent par le mot `new` :

In [21]:
df
  .withColumn("city_starts_with_new", $"city".startsWith("new"))
  .show()

+---------+--------+--------------------+
|superhero|    city|city_starts_with_new|
+---------+--------+--------------------+
|     thor|new york|                true|
|  aquaman|atlantis|               false|
|wolverine|new york|                true|
+---------+--------+--------------------+



La partie `$"city"` du code crée un objet Colonne.

Examinons les différentes façons de créer des objets Column.

### Instanciation des objets colonnes

Les objets colonnes doivent être créés pour exécuter les méthodes de colonne.
Un objet Colonne correspondant à la colonne city peut être créé en utilisant les trois syntaxes suivantes :

Les objets colonnes sont généralement passés en argument aux fonctions SQL (par exemple `upper($"city"))`. Nous allons créer des objets colonnes dans tous les exemples qui suivent.

#### gt

Créons un DataFrame avec une colonne de nombres entiers afin de pouvoir utiliser des méthodes de colonnes numériques.

In [22]:
val df = Seq(
    (10, "cat"), 
    (4, "dog"), 
    (7, null)
).toDF("num","word")

df: DataFrame = [num: int, word: string]

In [23]:
df.show

+---+----+
|num|word|
+---+----+
| 10| cat|
|  4| dog|
|  7|null|
+---+----+



Utilisons la méthode `gt()` (greater than) pour identifier toutes les lignes ayant un nombre supérieur à cinq:

In [24]:
df
  .withColumn("num_gt_5", col("num").gt(5))
  .show()

+---+----+--------+
|num|word|num_gt_5|
+---+----+--------+
| 10| cat|    true|
|  4| dog|   false|
|  7|null|    true|
+---+----+--------+



Nous pouvons également utiliser l'opérateur `>` pour effectuer des comparaisons "supérieures à":

In [25]:
df
  .withColumn("num_gt_5", col("num") >= 7)
  .show()

+---+----+--------+
|num|word|num_gt_5|
+---+----+--------+
| 10| cat|    true|
|  4| dog|   false|
|  7|null|    true|
+---+----+--------+



#### substr

Utilisons la méthode `substr()` pour créer une nouvelle colonne avec les deux premières lettres du mot colonne:

In [26]:
import org.apache.spark.sql.functions.col
df
  .withColumn("word_first_two", col("word").substr(0, 2))
  .show()

+---+----+--------------+
|num|word|word_first_two|
+---+----+--------------+
| 10| cat|            ca|
|  4| dog|            do|
|  7|null|          null|
+---+----+--------------+



import org.apache.spark.sql.functions.col


Notez que la méthode `substr()` renvoie `null` lorsqu'elle recoit null en entrée. Toutes les autres méthodes et fonctions SQL se comportent de la même manière (c'est-à-dire qu'elles renvoient `null` lorsque l'entrée est `null`).

Vos fonctions doivent traiter les entrées nulles avec élégance et renvoyer `null` lorsqu'elles recoivent `null` en entrée.

#### L'operatuer `+`

Utilisons l'opérateur `+` pour ajouter cinq à la colonne des nombres:

In [27]:
df
  .withColumn("num_plus_five", col("num").+(5))
  .show()

+---+----+-------------+
|num|word|num_plus_five|
+---+----+-------------+
| 10| cat|           15|
|  4| dog|            9|
|  7|null|           12|
+---+----+-------------+



Nous pouvons également sauter la notation par points lorsque nous invoquons la fonction:

In [28]:
df
  .withColumn("num_plus_five", col("num") + 5)
  .show()

+---+----+-------------+
|num|word|num_plus_five|
+---+----+-------------+
| 10| cat|           15|
|  4| dog|            9|
|  7|null|           12|
+---+----+-------------+



Le *syntactic sugar* rend plus difficile de voir que `+` est une méthode définie dans la classe Colonne.

#### lit

Utilisons la méthode `/` pour prendre `2` divisé par la colonne des `num`:

In [29]:
import org.apache.spark.sql.functions._
df
  .withColumn("two_divided_by_num", lit(2) / col("num"))
  .show()

+---+----+------------------+
|num|word|two_divided_by_num|
+---+----+------------------+
| 10| cat|               0.2|
|  4| dog|               0.5|
|  7|null|0.2857142857142857|
+---+----+------------------+



import org.apache.spark.sql.functions._


Notez que la fonction `lit()` doit être utilisée pour convertir 2 en un objet Column avant que la division puisse avoir lieu.

In [28]:
df
  .withColumn("two_divided_by_num", 2 / col("num"))
  .show()

cmd28.sc:3: overloaded method value / with alternatives:
  (x: Double)Double <and>
  (x: Float)Float <and>
  (x: Long)Long <and>
  (x: Int)Int <and>
  (x: Char)Int <and>
  (x: Short)Int <and>
  (x: Byte)Int
 cannot be applied to (org.apache.spark.sql.Column)
  .withColumn("two_divided_by_num", 2 / col("num"))
                                      ^Compilation Failed

: 

La méthode `/` est définie dans les deux classes Scala Int et Spark Column. Nous devons convertir le nombre en un objet Column, afin que le compilateur sache qu'il doit utiliser la méthode `/` définie dans la classe Spark Column. En analysant le message d'erreur, nous pouvons voir que le compilateur essaie par erreur d'utiliser l'opérateur `/` défini dans la classe Scala Int.

#### isNull

Utilisons la méthode `isNull` pour identifier quand la colonne word est `null`:

In [30]:
df
  .withColumn("word_is_null", col("word").isNull)
  .show()

+---+----+------------+
|num|word|word_is_null|
+---+----+------------+
| 10| cat|       false|
|  4| dog|       false|
|  7|null|        true|
+---+----+------------+



#### when / otherwise

Créons un DataFrame final avec les colonnes `word1` et `word2`, afin de pouvoir jouer avec les méthodes `===`, `when()`, et `otherwise()`:

In [31]:
val df = Seq( 
    ("bat", "bat"), 
    ("snake", "rat"), 
    ("cup", "phone"), 
    ("key", null)
).toDF("word1","word2")

df: DataFrame = [word1: string, word2: string]

In [32]:
df.show

+-----+-----+
|word1|word2|
+-----+-----+
|  bat|  bat|
|snake|  rat|
|  cup|phone|
|  key| null|
+-----+-----+



Ecrivons un petit algorithme de comparaison de mots qui analyse les différences entre les deux mots:

In [33]:
import org.apache.spark.sql.functions._
df
  .withColumn(
    "word_comparison",
    when($"word1" === $"word2", "same words")
      .when(length($"word1") > length($"word2"), "word1 is longer")
      .otherwise("i am confused")
  ).show()

+-----+-----+---------------+
|word1|word2|word_comparison|
+-----+-----+---------------+
|  bat|  bat|     same words|
|snake|  rat|word1 is longer|
|  cup|phone|  i am confused|
|  key| null|  i am confused|
+-----+-----+---------------+



import org.apache.spark.sql.functions._


`when()` et `otherwise()` sont la façon d'écrire la logique `if / else if / else` dans Spark.

Vous utiliserez tout le temps des méthode colonnes lorsque vous écrirez du code Spark.
Si vous n'avez pas de solides connaissances en programmation orientée objet, il peut être difficile de déterminer quelles méthodes sont définies dans la classe `Column` et quelles méthodes sont définies dans le paquet `org.apache.spark.sql.functions`.

## Introduction aux fonctions de Spark SQL

Cette section vous montre comment utiliser les fonctions SQL de Spark et comment construire vos propres fonctions SQL. Les fonctions Spark SQL sont essentielles pour presque toutes les analyses.

Les fonctions Spark SQL sont définies dans l'objet `org.apache.spark.sql.functions`. Il y a une tonne de fonctions ! (cf. Spark Doc)

La plupart des fonctions SQL prennent un ou des  argument(s) de type Column et renvoient des objets Column.

Montrons comment utiliser une fonction SQL. 
Créons un DataFrame avec une colonne `number` et utilisons la fonction `factorial` pour ajouter une colonne `number_factorial`:

In [ ]:
import org.apache.spark.sql.functions._

val df = Seq(2,3,4).toDF("number")
df
 .withColumn("number_factorial", factorial(col("number")))
 .show()

Si `Spark implicits` est importés (c'est-à-dire que vous avez lancé l'importation `spark.implicits._`), vous pouvez également créer un objet Column avec l'opérateur `$`. Ce code fonctionne également:

In [ ]:
import org.apache.spark.sql.functions._
import spark.implicits._

val df = Seq(2,3,4).toDF("number")

df
 .withColumn("number_factorial", factorial($"number"))
 .show()

Le reste de cette section se concentre sur les fonctions SQL les plus importantes qui seront utilisées dans la plupart des analyses.

#### lit() function

La fonction `lit()` crée un objet Column à partir d'une valeur littérale. Créons un DataFrame et utilisons la fonction `lit()` pour ajouter une colonne `wolof_hi` au DataFrame:

In [ ]:
import org.apache.spark.sql.functions.lit

val df = Seq("madeleine","anta","zakaria").toDF("word") 
df
 .withColumn("wolof_hi", lit("ziyaar"))
 .show()

La fonction `lit()` est particulièrement utile pour faire des comparaisons booléennes.

#### when() et otherwise()

Les fonctions `when()` et `otherwise()` sont utilisées pour le flux de contrôle dans Spark SQL, de la même manière que if et else dans d'autres langages de programmation.

Créons un DataFrame `contries` et utilisons des instructions `when()` pour ajouter une colonne  `continent`:

In [ ]:
val df = Seq("senegal","canada","italy","tralfamadore").toDF("country")
df
  .withColumn(
    "continent",
    when(col("country") === lit("senegal"), lit("africa"))
      .when(col("country") === lit("canada"), lit("north america"))
      .when(col("country") === lit("italy"), lit("europe"))
      .otherwise("not sure")
) .show()

Spark vous permet de couper parfois les appels à la méthode lit() et d'exprimer le code de manière compacte:

In [ ]:
df
  .withColumn(
    "continent",
    when(col("country") === "senegal", "africa")
      .when(col("country") === "canada", "north america")
      .when(col("country") === "italy", "europe")
      .otherwise("not sure")
) .show()

In [34]:
val df = Seq( 
    ("Amina", "Ba"), 
    ("Modou", "Ndiaye"), 
    ("Luiz", "Faye"), 
).toDF("Prenom","Nom")

df: DataFrame = [Prenom: string, Nom: string]

In [37]:
df
    .withColumn("fullName", concat(col("Prenom"),col("Nom")))
    .show()

+------+------+------------+
|Prenom|   Nom|    fullName|
+------+------+------------+
| Amina|    Ba|    Amina Ba|
| Modou|Ndiaye|Modou Ndiaye|
|  Luiz|  Faye|   Luiz Faye|
+------+------+------------+



La méthode when est définie à la fois dans la classe Column et l'objet functions. Chaque fois que vous voyez `when()` qui n'est pas précédé d'un point, c'est alors when de l'objet functions. `.when()` vient de la classe Column.

### Rédiger sa propre fonction SQL

Vous pouvez facilement créer vos propres fonctions SQL. Beaucoup de nouveaux développeurs Spark créent des fonctions définies par l'utilisateur alors qu'il serait beaucoup plus facile de créer simplement une fonction SQL personnalisée. Évitez les fonctions définies par l'utilisateur dans la mesure du possible !

Créons une fonction `lifeStage()` qui prend un argument `age` et renvoie "child", "teenager" ou "adult":

In [ ]:
import org.apache.spark.sql.Column

def lifeStage(col: Column): Column = {
    when(col < 13, "child")
    .when(col >= 13 && col <= 18, "teenager")
    .when(col > 18, "adult")
}

Voici comment utiliser la fonction lifeStage() :

In [ ]:
val df = Seq(10,15,25).toDF("age")

df
  .withColumn(
      "life_stage",
      lifeStage(col("age"))
  )
  .show()

In [ ]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.functions.col

def lifeStageString(colName: String): Column = {
    lifeStage(col(colName))
}

In [ ]:
val df = Seq(10,15,25).toDF("age")

df
  .withColumn(
      "life_stage",
      lifeStageString("age")
  )
  .show()

Créons une autre fonction qui enleve tous les espaces et met en majuscules tous les caractères d'une chaîne:

In [ ]:
import org.apache.spark.sql.Column

def trimUpper(col: Column): Column={
    trim(upper(col))
}

Lançons trimUpper() sur un échantillon de données:

In [ ]:
val df = Seq(
    "   some stuff",
    "like CHEESE ",
    "null"
).toDF("weird")

df
  .withColumn(
      "cleaned",
      trimUpper(col("weird"))
  )
  .show()

Des fonctions SQL personnalisées peuvent généralement être utilisées à la place des UDFs. Éviter les UDFs est un excellent moyen d'écrire un meilleur code Spark.

Les fonctions Spark SQL sont préférables aux UDFs parce qu'elles gèrent la cas null de maniere elegante (sans beaucoup de code) et parce qu'elles ne sont pas une boîte noire.

La plupart des analyses Spark peuvent être exécutées en utilisant la bibliothèque standard et en revenant à des fonctions SQL personnalisées si nécessaire. Évitez les UDFs à tout prix !

## Enchaînement de transformations de DataFrame

Cette section explique comment écrire des transformations de DataFrame et comment enchaîner des transformations multiples avec la méthode `Dataset#transform`.

### La méthode `transform` 

La méthode de `transform` de Dataset fournit une syntaxe concise pour l'enchaînement des transformations personnalisées.

Supposons que nous ayons une méthode `withGreeting()` qui ajoute une colonne `greeting` à un DataFrame et un `withFarewell()` methode qui ajoute une colonne `farewell` à un DataFrame:

In [ ]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.lit

def withGreeting(df: DataFrame): DataFrame = {
    df.withColumn("greeting", lit("hello world"))
}

def withFarewell(df: DataFrame): DataFrame = {
    df.withColumn("farewell", lit("goodbye"))
}

In [ ]:
val df = Seq( 
    "Aby", "Mohamed"
).toDF("someone")


val weirdDF = df
 .transform(withGreeting)
 .transform(withFarewell)

In [ ]:
weirdDF.show

La méthode transform peut facilement être chaînée avec les méthodes intégrées de Spark DataFrame, comme select:

In [ ]:
df
  .select("someone")
  .transform(withGreeting)
  .transform(withFarewell)

La méthode transform nous aide à écrire du code facile à suivre en évitant les appels de méthodes imbriquées. Sans la méthode transform, le code ci-dessus devient moins lisible :

In [ ]:
withFarewell(withGreeting(df)).show

ou pire encore:

In [ ]:
withFarewell(withGreeting(df)).select("someone").show

### La méthode `transform` avec arguments

Nos deux exemples de transformations precedentes (withFarewell et withGreeting) modifient les DataFrames de manière standard : c'est-à-dire qu'ils ajouteront toujours une colonne nommée farewell et greeting, chacune avec des valeurs codées en dur ("goodbye" et "hello world", respectivement).

Nous pouvons également créer des transformations de DataFrame personnalisées en définissant des transformations qui prennent des arguments. Pour cela, nous pouvons exploiter le currying avec des listes de paramètres multiples dans Scala.

Pour illustrer la différence, utilisons la même méthode `withGreeting()` que précédemment et ajoutons une méthode `withCat()` qui prend une chaîne de caractères comme argument:

In [ ]:
def withGreeting(df: DataFrame): DataFrame = {
    df.withColumn("greeting", lit("hello world"))
}

def withCat(name: String)(df: DataFrame): DataFrame = {
    df.withColumn("cats", lit(s"$name meow"))
}

Nous pouvons utiliser la méthode `transform` pour exécuter les méthodes `withGreeting()` et `withCat()`:

In [ ]:
val df = Seq( 
    "funny", 
    "person"
).toDF("something")

val niceDF = df
  .transform(withGreeting) 
  .transform(withCat("puffy"))

In [ ]:
niceDF.show()

## Effectuer des opérations sur plusieurs colonnes avec foldLeft

La méthode `foldLeft` de Scala peut être utilisée pour itérer sur une structure de données et effectuer de multiples opérations sur un Spark DataFrame.

Par exemple, foldLeft peut être utilisé pour éliminer tous les espaces dans plusieurs colonnes ou pour convertir tous les noms de colonnes d'un DataFrame en snake_case.

foldLeft est idéal lorsque vous souhaitez effectuer des opérations similaires sur plusieurs colonnes. Plongeons dans le vif du sujet !

### Revue de foldLeft sur Scala

Supposons que vous ayez une liste de trois nombres impairs et que vous souhaitiez calculer la somme de tous les nombres de la liste.

La méthode foldLeft permet d'itérer sur chaque élément de la liste et de garder une trace d'une somme courante:

In [ ]:
val odds = List(1,5,7)

In [ ]:
println {
    odds.foldLeft(0) {
        (memo: Int, num: Int) => memo + num 
    }
}

C'est la meme chose que la boucle suivante:

In [ ]:
var memo: Int = 0
for (num <- odds)  {
    memo += num // memo = memo + num
}

La fonction foldLeft est initialisée avec une valeur de départ de zéro et la somme courante est accumulée dans la variable mémo. Ce code additionne tous les nombres de la liste.

### Éliminer les espaces sur plusieurs colonnes

Créons un DataFrame et écrivons ensuite une fonction pour supprimer tous les espaces dans toutes les colonnes:

In [ ]:
val sourceDF = Seq(
    (" N e y m a r", "Brasil"), ("Sadio", "S e negal")
).toDF("name","country")

sourceDF.show

In [ ]:
val actualDF = Seq( 
    "name",
    "country"
).foldLeft(sourceDF) {
    (memoDF, colName) => memoDF.withColumn(
        colName,
        removeAllWhiteSpace(colName)
  )
}

def removeAllWhiteSpace(colName: String): Column = {
    regexp_replace(col(colName), "\\s+", "")
}

actualDF.show

## Introduction à la jointure  de diffusion avec Spark

Les jointures de diffusion (broadcast joins) sont parfaites pour relier un grand DataFrame à un petit DataFrame. Les jointures de diffusion ne peuvent pas être utilisées pour joindre deux grands DataFrames.

Cette section explique comment réaliser une simple jointure de diffusion et comment la fonction broadcast() aide Spark à optimiser le plan d'exécution.

### Concepte

Spark répartit les données sur les différents nœuds d'un cluster afin que plusieurs ordinateurs puissent traiter les données en parallèle. Les jointures traditionnelles sont difficiles avec Spark car les données sont réparties sur plusieurs machines.

Les liaisons de diffusion sont plus faciles à exécuter sur un cluster. Spark peut "diffuser" une petite DataFrame en envoyant toutes les données de cette petite DataFrame à tous les nœuds du cluster. Une fois le petit DataFrame diffusée, Spark peut effectuer une jointure sans un shuffling des données du grand DataFrame.

### Exemple

Créons un DataFrame avec des informations sur les personnes et un autre DataFrame avec des informations sur les villes. Dans cet exemple, les deux DataFrames seront petits, mais imaginons que le `peopleDF` est énorme et le `citiesDF` est minuscule:

In [ ]:
val peopleDF = Seq( 
    ("khadidiatou", "dakar"), 
    ("bally", "dakar"), 
    ("bayoulou", "bobo")
).toDF("first_name","city") 

peopleDF.show()

In [ ]:
val citiesDF = Seq(
    ("dakar", "senegal", 0.1),
    ("bobo", "burkina", 0.25)
).toDF("city","country","population")

La jointure simple des deux dataframes peut se faire en utilisant la methode `.join()`:

In [ ]:
peopleDF.join(
    citiesDF,
    peopleDF("city") <=> citiesDF("city"),
    "inner"
).show()

Diffusons maintenant citiesDF et joignons le avec peopleDF:

In [ ]:
peopleDF.join(
    broadcast(citiesDF),
    peopleDF("city") <=> citiesDF("city")
).show()

L'opérateur d'égalité Spark "null safe" (`<=>`) est utilisé pour effectuer cette jointure.

### Analyse des plans physiques des jointures

Utilisons la méthode `explain()` pour analyser le plan physique de la jointure de diffusion:

In [ ]:
peopleDF.join(
  broadcast(citiesDF),
  peopleDF("city") <=> citiesDF("city")
).explain()

Dans cet exemple, Spark est assez intelligent pour renvoyer le même plan physique, même lorsque la méthode `broadcast()` n'est pas utilisée:

In [ ]:
peopleDF.join(
  citiesDF,
  peopleDF("city") <=> citiesDF("city")
).explain()

### Élimination de la colonne `city` en double

Nous pouvons passer une séquence de colonnes en argument pour supprimer automatiquement la colonne en double:

In [ ]:
peopleDF.join( 
    broadcast(citiesDF), 
    Seq("city")
).show()

Examinons le plan physique généré par ce code:

In [ ]:
peopleDF.join( 
    broadcast(citiesDF), 
    q("city")Se
).explain()

Un code qui renvoie le même résultat sans s'appuyer sur la séquence jointe:

In [ ]:
peopleDF.join(
    broadcast(citiesDF),
    peopleDF("city") <=> citiesDF("city")
)
.drop(citiesDF("city"))
.explain()

Il est préférable d'éviter le raccourci "join syntax" pour que vos plans physiques restent aussi simples que possible.

Vous pouvez passer a la méthode `explain()` un argument `true` pour voir le plan logique parsé, le plan logique analysé et le plan logique optimisé en plus du plan physique:

In [ ]:
peopleDF.join(
  broadcast(citiesDF),
  peopleDF("city") <=> citiesDF("city")
)
.drop(citiesDF("city"))
.explain(true)

Remarquez comment les plans logiques parsés, analysés et optimisés contiennent tous `ResolvedHint (broadcast)` car la fonction broadcast() a été utilisée. Cet indice n'est pas inclus lorsque la fonction broadcast() n'est pas utilisée:

In [ ]:
peopleDF.join(
  citiesDF,
  peopleDF("city") <=> citiesDF("city")
)
.drop(citiesDF("city"))
.explain(true)

Les jointures de diffusion sont un excellent moyen d'ajouter aux grands DataFrames des données stockées dans des fichiers de données de vérité relativement petits et provenant d'une seule source. Des DataFrames allant jusqu'à 2 Go peuvent être diffusés.